II. Build minimum spanning tree  
1. Calculate the correlation matrix of price change
2. Obtain the edge list by Prim's algorithm of minimum spanning tree
3. Creat the Graph object by networkingx  

Schematising of Minimum Spanning Tree:
1. rank a couple of vertices(stocks) from the nearest to the farthest  
2. draw the first edge from this rank  
3. continue in the rank  
4. if the new edge does not close a cycle draw it  
5. go to point 3  
6. stop when all the vertices have been drawn  

Reference:  
https://mktstk.com/2015/03/04/stock-market-visualization-minimum-spanning-trees/  
http://peekaboo-vision.blogspot.com/2012/02/simplistic-minimum-spanning-tree-in.html  

In [1]:
import numpy as np
import pandas as pd
import networkx as nx

In [3]:
# Read in correlation matrix
corr = pd.read_csv('corr_2016.csv', index_col=0)

In [4]:
# Calculate the distance between pairwise stocks
# distance[i, j] = sqrt(2.0 * ( 1 – correlation[i, j] ) )
dist = 1- abs(corr) # adjusted 
dist.to_csv('dist_2016.csv')

In [5]:
# Prim's algotithm to get edge list by minimum spanning trees
def minimum_spanning_tree(X, copy_X=True):
    """X are edge weights of fully connected graph"""
    if copy_X:
        X = X.copy()
 
    if X.shape[0] != X.shape[1]:
        raise ValueError("X needs to be square matrix of edge weights")
    n_vertices = X.shape[0]
    spanning_edges = []
     
    # initialize with node 0:                                                                                         
    visited_vertices = [0]                                                                                            
    num_visited = 1
    # exclude self connections:
    diag_indices = np.arange(n_vertices)
    X[diag_indices, diag_indices] = np.inf
     
    while num_visited != n_vertices:
        new_edge = np.argmin(X[visited_vertices], axis=None)
        # 2d encoding of new_edge from flat, get correct indices                                                      
        new_edge = divmod(new_edge, n_vertices)
        new_edge = [visited_vertices[new_edge[0]], new_edge[1]]                                                       
        # add edge to tree
        spanning_edges.append(new_edge)
        visited_vertices.append(new_edge[1])
        # remove all edges inside current tree
        X[visited_vertices, new_edge[1]] = np.inf
        X[new_edge[1], visited_vertices] = np.inf                                                                     
        num_visited += 1
    return np.vstack(spanning_edges)

In [ ]:
# Get edges
dist = np.array(dist)
edge_list_raw = minimum_spanning_tree(dist)
edge_list = []
for i, j in edge_list_raw:
    edge = (tickers[i], tickers[j])
    edge_list.append(edge)

# Export edges
edges = pd.DataFrame(edge_list)
edges.to_csv('edges_2016.csv')

In [ ]:
# Create graph object by adding edges
mst = nx.Graph()
for i, j in edge_list:
    mst.add_edge(i,j)
    
# Add sector information to nodes
for node in mst.nodes():
    mst.node[node]['attribute']=sp500_dic[node]
    
# Export network for further use in Gephi
nx.write_graphml(mst, 'sp500_2016.graphml')